# PDAs and context-free grammars

## Context Free Pumping Lemma

:::{prf:theorem} Pumping Lemma for CFLs
:name: toc-cfl-pumping-lemma

For every CFL $A$, there exists an integer $p$ such that if $s \in A$ and $|s| \geq p$, then $s = uvxyz$ where

1. $uv^ixy^iz \in A$ for all $i \geq 0$,
2. $vy \neq \epsilon$,
3. $|vxy| \leq p$.

:::

::::{prf:example} A non-context-free language


The language $A = \{0^k1^k2^k | k \geq 0\}$ is not a CFL.

:::{dropdown} Proof

Suppose $A = \{0^k1^k2^k | k \geq 0\}$ is CFL.
Then by the {ref}`pumping lemma for CFLs<toc-cfl-pumping-lemma>`, it has a pumping length $p$.
Let $s = 0^p 1^p 2^p$.
By the pumping lemma, it holds that $s = uvxyz$, such that the pumping lemma conditions hold.
By the condition $|vxy| \leq p$, the substring $vxy$ cannot both contain both zeros andd twos.
Therefore, $uv^ixy^iz$ will not contain equal numbers of zeros and twos when $i \geq 2,$ which contradicts the pumping lemma.
So $A$ is not a CFL.
:::
::::

::::{prf:lemma} CFL not closed under intersection

The class of context-free languages are not closed under intersection.

:::{dropdown} Proof

The languages $A_1 = \{0^k 1^k 2^l | k, l \geq 0\}$ and $A_2 = \{0^l 1^k 2^k | k, l \geq 0\}$ are context free since we can build PDAs that recognise them.
For example, for $A_1$ we can construct a PDA which pushes the zeros on its stack, then pops them as it reads the ones, and lastly read any number of twos.
Similarly we can build a PDA that recognises $A_2$.
However, $A_1 \cap A_2 = \{0^k 1^k 2^k | k \geq 0 \}$, which is not a CFL, so the class of CFLs is not closed under intersection.
:::
::::